In [5]:
from app import App

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
SYMBOLS = ["BTCUSDT", "ETHUSDT", "QNTUSDT"]
INTERVAL = "1d"

backtest = App(symbols = SYMBOLS, interval = INTERVAL, start="2023-01-02", end = "2023")
backtest.run()

In [7]:
backtest.results()

 ==== BTCUSDT ==== 
                  LS  LS_reduit      long     short
total_PL    1.188554   1.188554  1.274084  0.932870
nb_trades   0.000000   0.000000  0.000000  0.000000
min_ret    -0.038587  -0.039714  0.000000  0.000000
quartile_1 -0.037897  -0.039677  0.000000  0.000000
median     -0.037206  -0.039640  0.000000  0.000000
mean        1.004556   1.012415  0.000000  0.000000
quartile_3 -0.036515  -0.039604  0.000000  0.000000
max_ret     0.075724   0.361824  0.000000  0.000000
drawdown   -0.272767  -0.271142 -0.270837 -0.067520
skew        1.088527   3.161578  0.000000  0.000000
kurt        1.639806   8.426064  0.000000  0.000000
VaR_hist    0.038449   0.039706  0.038570  0.039687
VaR-gauss   0.071059   0.000000  0.000000  0.000000
CVaR        0.001528   0.010268  0.009318  0.011375
 ==== ETHUSDT ==== 
                  LS  LS_reduit      long     short
total_PL    0.971054   0.971054  1.128876  0.860195
nb_trades   0.000000   0.000000  0.000000  0.000000
min_ret    -0.069261  -0

In [8]:
DataLS1 = backtest.output["BTCUSDT"][0]
LS_reduit = backtest.output["BTCUSDT"][1]

In [9]:
backtest.plot("BTCUSDT")

In [10]:
backtest.plot("ETHUSDT")
backtest.plot("QNTUSDT")

In [11]:
x

NameError: name 'x' is not defined

In [ ]:
backtest.plot_returns("BTCUSDT")

In [ ]:
df = backtest.dataLS.copy()
df_r = backtest.LS_reduit

In [ ]:
import pandas as pd
import numpy as np

data = pd.DataFrame(index = range(1, 31+1))

In [ ]:
df_nov = df_r.loc[df_r.date.dt.month == 11][['date', 'returns']]
df_nov['date'] = df_nov.date.dt.day
df_nov = df_nov.groupby('date').sum()
df_nov.rename(columns = {'returns': 'november'}, inplace = True)

In [ ]:
df_dec = df_r.loc[df_r.date.dt.month == 12][['date', 'returns']]
df_dec['date'] = df_dec.date.dt.day
df_dec = df_dec.groupby('date').sum()
df_dec.rename(columns = {'returns': 'december'}, inplace = True)

In [ ]:
dd = pd.concat([data, df_nov, df_dec], axis=1)
dd.T

In [ ]:
import plotly.express as px

px.imshow(dd.T)

# Compare

In [ ]:
backtest.data_1.head(25)[['return', 'log_return', 'momentum']]

In [ ]:
col = ['price', 'status', 'amount', 'quantity', 'value', 'returns', 'cum_ret_per_trade', 'cum_ret', 'price_cum_ret']
col_r = ['price', 'status', 'amount', 'quantity', 'value', 'returns', 'cum_ret', 'price_cum_ret']

data_LS1 = backtest.output["BTCUSDT"][0][col]
LS_reduit = backtest.output["BTCUSDT"][1][col_r]


import sqlalchemy
import pandas as pd

path = "C:/Users/cc/Desktop/CedAlgo/AlgoTrading/data/trade/"
engine = sqlalchemy.create_engine('sqlite:///'+path+'BTCUSDT_LS.db')

data_LS2 = pd.read_sql('DataLS', engine)[['date']+col]
LS_reduit2 = pd.read_sql('LS_reduit', engine)[col_r]

In [ ]:
data_LS1.head(25)

In [ ]:
data_LS2.head(25)

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# make subplot
def subplots(nb_rows , nb_cols, row_heights = None, column_widths = None, vertical_spacing=0.01, horizontal_spacing = 0.02):
    
    fig = make_subplots(rows = nb_rows, cols = nb_cols, shared_xaxes = True, shared_yaxes = False,
                        row_heights = row_heights, column_widths= column_widths,
                        vertical_spacing = vertical_spacing, horizontal_spacing = horizontal_spacing)
    return fig


# Plot candle
def plot_candle(fig, col, row, data): 
    fig.add_trace(
        data = go.Candlestick(
                    x = data.index , open = data.open, close = data.close,
                    high = data.high, low = data.low, name = data.symbol[0]
                ),
        layout = go.Layout(
                height = 500 , width = 1000,
                template="simple_white"
                ),
        col = col, row = row
    )

def add_line(fig, col, row, data, name, color = None):
    fig.add_trace(
        data = go.Scatter(
                    x = data.index,
                    y = data[name],
                    name = name
                ),
        col = col, row = row
    )


def add_scatter(fig, col, row, data, name, color):
    fig.add_trace(
        data = go.Scatter(
                    x = data.index,
                    y = data[name],
                    mode = 'markers',
                    marker_color = color,
                    name = name
                ),
        col = col, row = row
    )


# signal point
def signal_point(fig, col, row, x, y, name, marker, size = 10):
    fig.add_trace(
        data = go.Scatter(
                    x = x,
                    y = y,
                    mode = "markers",
                    marker_symbol = marker["symbol"],
                    marker_size = marker["size"],
                    marker_color = marker["color"],
                    name = name
                ),
        col = col , row = row
    )
    
    
# Plot long short brenchmank
def plot_LSB(fig, col , row ,Benchmark, Trades, Trades_long, Trades_short):
    fig.add_trace(
        data = go.Scatter(
                    x = Benchmark.index,
                    y = Benchmark["price_cum_ret"],
                    marker_color = "pink",
                    name = "price"
                ),
        col = col , row = row
    )
    fig.add_trace(
        data = go.Scatter(
                    x = Trades.index,
                    y = Trades["cum_ret"],
                    marker_color = "green",
                    name = "trade"
                ),
        col = col , row = row
    )
    fig.add_trace(
        data = go.Scatter(
                    x = Trades_long.index,
                    y = Trades_long["cum_ret"],
                    marker_color = "blue",
                    name = "long"
                ),
        col = col , row = row
    )
    fig.add_trace(
        data = go.Scatter(
                    x = Trades_short.index,
                    y = Trades_short["cum_ret"],
                    marker_color = "red",
                    name = "short"
                ),
        col = col , row = row
    )



def color_trades(fig, col, row, status, x , y, color, opacity):
    '''
    x : tuple , x = (x0, x1)
    y : tuple , y = (y0, y1)
    '''
    fig.add_shape(x0 = x[0], x1 = x[1],
                  y0 = y[0]*0.6, y1 = y[1]*1.4,
                  fillcolor = color, opacity = opacity,
                  col = col , row = row)

def color_returns(fig, col, row, status, x , color, opacity):
    
    fig.add_vrect(x0 = x[0], x1 = x[1],
                  fillcolor = color, opacity = opacity,
                  col = col , row = row)


def plot_return(fig, col, row, dataLS, LS_reduit, LS_long, LS_short):
    fig.add_trace(
        go.Scatter(
            x = dataLS.index, y = dataLS["cum_ret"],
            name = "LS"
        )
    )
    fig.add_trace(
        go.Scatter(
            x = dataLS.index, y = dataLS["price_cum_ret"],
            name = "price"
        ),
        col = col, row = row
    )
    fig.add_trace(
        go.Scatter(
            x = LS_reduit.index, y = LS_reduit["cum_ret"],
            name = "LS_reduit"
        ),
        col = col, row = row
    )
    fig.add_trace(
        go.Scatter(
            x = LS_long.index, y = LS_long["cum_ret"],
            name = "LS_long"
        ),
        col = col, row = row
    )
    fig.add_trace(
        go.Scatter(
            x = LS_short.index, y = LS_short["cum_ret"],
            name = "LS_short"
        ),
        col = 1, row = 1
    )
    